In [ ]:
# python commonFriend.py soc-LiveJournal1Adj.txt

from __future__ import print_function
import sys
from pyspark import SparkContext

def mapper(line):
    user = line[0]
    friends = line[1].split(',')
    out = []

    for friend in friends:
        if len(friend) > 0:
            key = ','.join(sorted([user, friend]))
            val = set(friends)
            out.append((key, val))
    return out

if __name__ == "__main__":
    if len(sys.argv) != 2:
        print("Usage: wordcount <file>", file=sys.stderr)
        sys.argv[1] = "soc-LiveJournal1Adj.txt"
        # sys.exit(-1)
    sc = SparkContext.getOrCreate()
    result = sc.textFile(sys.argv[1]).map(lambda x: x.split('\t'))\
            .flatMap(mapper)\
            .reduceByKey(lambda l1, l2: l1.intersection(l2)).sortByKey()\
            .filter(lambda out: len(out[1]) > 0)\
            .map(lambda x: x[0]+"\t"+','.join(sorted(x[1])))
    result.repartition(1).saveAsTextFile("Q1_Output")
    
    

    

In [ ]:
pets = sc.parallelize([("cat",1),("dog", 1),("cat", 2)])
reduced = pets.reduceByKey(lambda x, y: x + y)
print(reduced.collect())
grouped = pets.groupByKey().mapValues(list)
#[('cat', [1, 2]), ('dog', [1])] =>[1, 2] is iterable obj need to be map
# .mapValues(list) == .map(lambda x : (x[0], list(x[1])))
print(grouped.collect())
#print(pets.sortByKey().collect())

In [ ]:
from __future__ import print_function
import sys
from pyspark import SparkContext

sc = SparkContext.getOrCreate();
lines = sc.parallelize([("to be or"),("not to be")])
counts = lines.flatMap(lambda line: line.split(" ")).map(lambda word : (word, 1)).reduceByKey(lambda x, y: x + y)
print(counts.collect())

In [ ]:
def mapper(line):
    key = line[0]
    val = []
    
    for i in line[1]:
        val.append(list(i))
            
    
    return (key,val)

visits = sc.parallelize([("index.html", "1.2.3.4"),("about.html", "3.4.5.6"),("index.html", "1.3.3.1") ])
pageNames = sc.parallelize([ ("index.html", "Home"),("about.html", "About") ])

print(visits.join(pageNames).collect())
print(visits.cogroup(pageNames).map(mapper).collect())




In [ ]:
# python yelpUserRate.py business.csv review.csv user.csv

from __future__ import print_function
import sys
from pyspark import SparkContext


if __name__ == "__main__":
    if len(sys.argv) != 4:
        print("Usage: wordcount <business.csv> <review.csv> <user.csv>", file=sys.stderr)
        sys.argv = [0,0,0,0]
        sys.argv[1] = "business.csv"
        sys.argv[2] = "review.csv"
        sys.argv[3] = "user.csv"
        #sys.exit(-1)


    sc = SparkContext.getOrCreate();
    #"business_id"::"full_address"::"categories"
    business = sc.textFile(sys.argv[1]).map(lambda x: (x.split('::')[0], x.split('::')[1]))\
            .filter(lambda out: "Stanford" in out[1])#business_id, full_address
    #"review_id"::"user_id"::"business_id"::"stars"
    review = sc.textFile(sys.argv[2]).map(lambda x: (x.split('::')[2], (x.split('::')[1], x.split('::')[3])))
    #business_id, user_id, stars
    
    #"user_id"::"name"::"url"
    #user = sc.textFile(sys.argv[3])
    
    
    #business_id, (full_address, (user_id, stars))
    join = business.join(review)
    print(join.collect()[0:1])
    #user_id stars
    result = join.map(lambda x: '\t'.join(x[1][1]))
    print(result.collect()[0:1])
    #result.repartition(1).saveAsTextFile("Q3_output")
    

In [ ]:
print(business.collect())

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split
from pyspark.sql.functions import udf, desc
from pyspark.sql.types import IntegerType
from pyspark import SparkContext, SparkConf

conf = SparkConf().setMaster("local").setAppName("sql_sample")
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

# data is in the same folder of this script
# load data 
review = sc.textFile("review.csv").map(lambda line: line.split("::")).toDF()
review = review.select(review._1.alias('review_id'), review._2.alias('user_id'), review._3.alias('business_id'), review._4.alias('stars'))



#review = review.select('business_id', 'user_id').groupBy('business_id').count()
#review = review.filter("`count` >= 10").orderBy('count', ascending=False)
review.show()

In [ ]:
documentDF = spark.createDataFrame([
    ("Hi I heard about Spark".split(" "), ),
    ("I wish Java could use case classes".split(" "), ),
    ("Logistic regression models are neat".split(" "), )
], ["text"])

# Learn a mapping from words to Vectors.
word2Vec = Word2Vec(vectorSize=3, minCount=0, inputCol="text", outputCol="result")
model = word2Vec.fit(documentDF)

result = model.transform(documentDF)
for row in result.collect():
    text, vector = row
    print("Text: [%s] => \nVector: %s\n" % (", ".join(text), str(vector)))

In [ ]:
from pyspark.ml.feature import PCA
from pyspark.ml.linalg import Vectors

data = [(Vectors.sparse(5, [(1, 1.0), (3, 7.0)]),),
        (Vectors.dense([2.0, 0.0, 3.0, 4.0, 5.0]),),
        (Vectors.dense([4.0, 0.0, 0.0, 6.0, 7.0]),)]
df = spark.createDataFrame(data, ["features"])

pca = PCA(k=3, inputCol="features", outputCol="pcaFeatures")
model = pca.fit(df)

result = model.transform(df).select("pcaFeatures")
result.show(truncate=False)

In [ ]:
from pyspark.ml.feature import DCT
from pyspark.ml.linalg import Vectors

df = spark.createDataFrame([
    (Vectors.dense([0.0, 1.0, -2.0, 3.0]),),
    (Vectors.dense([-1.0, 2.0, 4.0, -7.0]),),
    (Vectors.dense([14.0, -2.0, -5.0, 1.0]),)], ["features"])

dct = DCT(inverse=False, inputCol="features", outputCol="featuresDCT")

dctDf = dct.transform(df)

dctDf.select("featuresDCT").show(truncate=False)

In [ ]:
from pyspark.ml.feature import PolynomialExpansion
from pyspark.ml.linalg import Vectors

df = spark.createDataFrame([
    (Vectors.dense([2.0, 1.0]),),
    (Vectors.dense([0.0, 0.0]),),
    (Vectors.dense([3.0, -1.0]),)
], ["features"])

polyExpansion = PolynomialExpansion(degree=3, inputCol="features", outputCol="polyFeatures")
polyDF = polyExpansion.transform(df)

polyDF.show(truncate=False)